In [2]:
from dotenv import load_dotenv
import os
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
import hashlib
from pinecone import Pinecone
from langchain_openai import OpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.output_parsers import StrOutputParser
from sklearn.metrics.pairwise import cosine_similarity
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore
from prompt_templates import prompt_templates
from langchain_core.prompts import MessagesPlaceholder
from google.cloud.firestore_v1.base_query import FieldFilter

c:\Codes\Django\thesis_django\.backend_venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv()

# Firestore Initialization
credential_path = r'C:\Codes\Django\thesis_django\echo_backend\echo_chatbot\ServiceAccountKey.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credential_path

if not firebase_admin._apps:
    cred = credentials.Certificate(r'C:\Codes\Django\thesis_django\echo_backend\echo_chatbot\ServiceAccountKey.json')
    firebase_admin.initialize_app(cred)

try:
    db = firestore.Client()
    print("*Firestore connected successfully!")
except Exception as e:
    print(f"Failed to connect to Firestore: {e}")

# API Keys Initialization
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

if not OPENAI_API_KEY:
    print("OpenAI API Key not found!")
if not PINECONE_API_KEY:
    print("Pinecone API Key not found!")

# Pinecone Initialization
try:
    pc = Pinecone(api_key=PINECONE_API_KEY)
    print("*Pinecone connected successfully!")
except Exception as e:
    print(f"Failed to connect to Pinecone: {e}")


# OpenAI Initialization
try:
    client=OpenAI(api_key=OPENAI_API_KEY)
    LLM = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")
    EMBEDDINGS = OpenAIEmbeddings(model='text-embedding-3-small')
    print("*OpenAI connected successfully!")
except Exception as e:
    print(f"Failed to connect to OpenAI: {e}")

*Firestore connected successfully!
*Pinecone connected successfully!
*OpenAI connected successfully!


In [4]:
query = "What are the keypoints of the project meeting"
user_id = "WuhmTzwTwmerjkSSK4XT8FyJS263"
session_id = "session1"
organization = "SCS"

In [5]:
# Get Embeddings
def get_embeddings(text):
    """
    This function returns a list of the embeddings for a given query
    """
    text_embeddings = EMBEDDINGS.embed_query(text)
    # print("Generating Embeddings: Done!")
    return text_embeddings

query_embeddings = get_embeddings(text=query)
print(query_embeddings)
type(query_embeddings)

[-0.040968623012304306, 0.06611032783985138, 0.07803546637296677, -0.0123166898265481, -0.04720596224069595, 0.002129489090293646, 0.0016022687777876854, 0.015428491868078709, 0.03176373243331909, -0.020470572635531425, 0.06138423830270767, -0.04577714577317238, -0.04462309926748276, -0.05099782720208168, -0.018959321081638336, -0.051025304943323135, -0.02467459626495838, -0.023012220859527588, -0.003561742138117552, 0.0037059979513287544, 0.02103385701775551, 0.030829505994915962, -0.020182060077786446, 0.02735362946987152, -0.05275637283921242, 0.018327344208955765, -0.029318256303668022, -0.03786369040608406, 0.04541993886232376, 0.04204023629426956, 0.00833935383707285, -0.04000691697001457, -0.013759247027337551, 0.02126741223037243, -0.028026822954416275, 0.04918432608246803, -0.009637654758989811, -0.006388467270880938, -0.03374209627509117, -0.014411832205951214, 0.005028341896831989, -0.014178275130689144, -0.02189939096570015, 0.0018701722146943212, 0.021075071766972542, -0.0

list

In [6]:
def resolve_namespace(query_embeddings, organization):
    """
    Resolves the namespace by either selecting the most similar one
    """
    def fetch_summaries_by_organization(organization):
        """
        Fetches summaries by organization
        """
        summaries = {}
        meetings_ref = db.collection("Meetings")
        query = meetings_ref.where(filter=FieldFilter("organization", "==", organization))
        docs = query.stream()

        for doc in docs:
            data = doc.to_dict()
            meeting_title = data.get("meetingTitle")
            summary = data.get("meetingSummary")
            if meeting_title and summary:
                summaries[meeting_title] = summary
        
        print(f"Fetched summaries for organization '{organization}': {summaries}")
        return summaries

    def get_most_similar_namespace(query_embeddings, summaries):
        """
        Rank namespaces by semantic similarity to the query.
        """
        
        summary_embeddings = {title: get_embeddings(summary) for title, summary in summaries.items()}
        print("Generated summary embeddings:", summary_embeddings)

        similarities = {
            title: cosine_similarity([query_embeddings], [embedding])[0][0] for title, embedding in summary_embeddings.items()
        }

        print("Computed similarities:", similarities)

        ranked_namespaces = sorted(similarities.items(), key=lambda x: x[1], reverse=True)
        print("Ranked namespaces:", ranked_namespaces)
        
        return ranked_namespaces[0][0]
    
    summaries = fetch_summaries_by_organization(organization)

    namespace = get_most_similar_namespace(query_embeddings, summaries)
    print(f"Selected namespace: {namespace}")
    return namespace

meeting_title = resolve_namespace(query_embeddings=query_embeddings, organization=organization)
print(meeting_title)
type(meeting_title)

Fetched summaries for organization 'SCS': {'Kickoff Meeting': 'On January 15, 2024, a kickoff meeting was held for a new software development project focused on creating a customer management system. Participants included John (Project Manager), Alice (Lead Developer), Bob (UI/UX Designer), and Sara (QA Analyst). The team discussed the project scope, which includes managing customer data, tracking interactions, and generating reports, using a microservices architecture with Java, React, and PostgreSQL. The timeline is set over six months with phases for planning and design, development, testing, and deployment. Responsibilities were outlined, with Alice overseeing development, Bob handling design, Sara managing QA, and John coordinating the project. Regular bi-weekly check-ins will be conducted to ensure deadlines are met and address any issues promptly.', 'Project Meeting': 'During the project meeting on January 9, 2025, led by Czech, the team discussed the final preparations for the 

str

In [10]:
# Get Relevant Documents
def query_pinecone_index(query_embeddings, meeting_title, index, top_k=5, include_metadata=True):
    """
    Query a Pinecone index.
    """
    # Build filter conditions directly for Pinecone
    filter_conditions = {}

    # Include date and meeting title if specified
    if meeting_title.lower() != 'unknown':
        filter_conditions['title'] = meeting_title

    # Query Pinecone using the build filter conditions
    query_response = index.query(
        vector=query_embeddings,
        filter=filter_conditions,
        top_k=top_k,
        include_metadata=include_metadata,
        namespace=meeting_title )

    print("Querying Pinecone Index: Done!")
    return [match['metadata']['text'] for match in query_response['matches']], [match['metadata']['date'] for match in query_response['matches']], [match['metadata']['title'] for match in query_response['matches']]

index = pc.Index(organization.lower())
text_answers, dates, titles = query_pinecone_index(query_embeddings=query_embeddings, meeting_title=meeting_title, index=index)
print(f"{text_answers}\n{dates[0]}\n{titles[0]}")
type(text_answers)
type(dates)
type(titles)

Querying Pinecone Index: Done!
["Czech: Hello my name is Czech.\nGian: Hello my name is Gian.\nShaundyl: Hello my name is Shaundyl.\nCzech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?", 'Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the common issues we’re anticipating. We’ll also monitor social media for any unexpected feedback.\nCzech: Sounds like we’re in good shape. Thanks, everyone. Let’s aim to regroup tomorrow for a final status check. Anything else before we wrap up?\nShaundyl: Nothing from my side. I’ll update you if any blockers come up.\nGian: I’m all set. Let’s get this done!', 'Czech: Great to hear that. Gian, how are we looking on the project timeline? Any changes or concerns?\nGian (Project Manager): We’re on track, but barely.

list

In [37]:
def initialize_chat_history(user_id, session_id):
    """
    Initializes a chat history object.
    """
    chat_history = []
    doc_ref = db.collection("chatHistory").document(user_id).collection("session").document(session_id)
    doc_snapshot = doc_ref.get()
    try:
        if doc_snapshot.exists:
            messages = doc_snapshot.get('messages')
            if messages is None:
                print(f"No 'messages' field found in document for user_id={user_id}, session_id={session_id}")
                return chat_history
            messages = doc_snapshot.get('messages')

            for message in messages:
                chat_history.append(message)
            print(f"Chat History Initialized: {chat_history}")
        else:
            print(f"No document found for user_id={user_id}, session_id={session_id}")
    except Exception as e:
        print(f"Error initializing chat history: {str(e)}")
    
    return chat_history

def update_chat_history(user_id, session_id, chat_history):
    """
    Updates the chat history object.
    """
    doc_ref = db.collection("chatHistory").document(user_id).collection("session").document(session_id)
    try:
        doc_ref.update({
            'messages': chat_history
        })
    except Exception as e:
        print(f"Error updating chat history: {str(e)}")

def process_chat_history(chat_history):
    """
    Changes the chat history list into a HumanMessages and AIMessages Schema
    """
    process_chat_history = []
    for idx, message in enumerate(chat_history):
        if idx % 2 == 0:
            process_chat_history.append(HumanMessage(message))
        else:
            process_chat_history.append(AIMessage(message))

        
    return process_chat_history

chat_history = initialize_chat_history(user_id=user_id, session_id=session_id)

Chat History Initialized: []


In [39]:
def decomposition_query_process(question, text_answers, chat_history):
    """Implements decomposition query"""

    def output_parser(output):
        """
        Helps parses the LLM output, prints it, and returns it.
        """
        print("\n" + output.content + "\n")

        return output.content

    def decompose_question(question):
        """
        Decomposes a complex question into smaller questions.
        """
        prompt = prompt_templates.decomposition_template().format(question=question)
        response = LLM.invoke(prompt)
        subquestions = response.content.split("\n")
        print("Decomposing Question: Done!")

        return subquestions
    
    def generate_qa_pairs(subquestions, context):
        """Generates QA pairs by answering each subquestion."""
        qa_pairs = []
        for subquestion in subquestions:
            context = context
            rag_prompt = prompt_templates.qa_template().format(context=context, subquestion=subquestion)
            answer = LLM.invoke(rag_prompt)
            qa_pairs.append((subquestion, answer))
        print("Generating QA Pairs: Done!")

        return qa_pairs
    
    def build_final_answer(question, context, qa_pairs):
        """Builds a final answer by integrating the context and QA pairs."""
        qa_pairs_str = "\n".join([f"Q: {q}\nA: {a}" for q, a in qa_pairs])
        # final_prompt = prompt_templates.final_rag_template().format(context=context, qa_pairs=qa_pairs_str, question=question)
        final_prompt = prompt_templates.final_rag_template_with_memory().format(context=context, qa_pairs=qa_pairs_str, question=question, chat_history=chat_history)
        final_response = LLM.invoke(final_prompt)
        print("Building Final Answer: Done!")

        return final_response
    
    subquestions = decompose_question(question)
    qa_pairs = generate_qa_pairs(subquestions, text_answers)
    print(qa_pairs)
    final_answer = build_final_answer(question, text_answers, qa_pairs)

    return output_parser(final_answer)

response = decomposition_query_process(question=query, text_answers=text_answers, chat_history=chat_history)
print(response)
type(response)

Decomposing Question: Done!
Generating QA Pairs: Done!
[('1. What is the main objective of the project meeting?', AIMessage(content='The main objective of the project meeting is to go over the final details before the product launch, ensuring that all aspects of the project, including development, support, and marketing, are on track and any potential issues are addressed promptly to meet the launch date successfully.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 441, 'total_tokens': 494, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_7c63087da1', 'finish_reason': 'stop', 'logprobs': None}, id='run-c9bb51b4-0a8f-41c0-b450-1a728ee2d26d-0', usage_metadata={'input_tokens': 441, 'output_tokens': 53, 

str

In [ ]:
print(f"Query: {query}")
print(f"Relevant Document: {text_answers}")
print(f"Answer: {response}")

In [40]:
sample_queries = [
    "Who were the attendees of the project meeting",
    "What were the decisions made regarding project meeting",
    "Find discussions related to marketing in project meeting",
    "Who is the team lead of project meeting",
    "What suggestions were made by Shaundyl?"
]

expected_responses = [
    "The attendees of the project meeting were Czech, Gian, and Shaundyl",
    "Czech scheduled another meeting for the following day to conduct a final status check",
    "We’re on track, but barely. The marketing materials were delayed by two days, but I managed to align the social media schedule to compensate for the delay. As long as the development and testing stay on track, we should meet the launch date. We might want to allocate some buffer time for any last-minute issues though.",
    "The team lead of project meeting is Czech",
    "Shaundyl planned to complete bug fixes by midday the following day to allow QA to start testing immediately afterward, providing a 24-hour window for testing before the product launch."
]

In [41]:
dataset = []

for query,reference in zip(sample_queries,expected_responses):

    text_answers = query_pinecone_index(query_embeddings=query_embeddings, meeting_title=meeting_title, index=index)
    response = decomposition_query_process(question=query, text_answers=text_answers, chat_history=[])
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":text_answers,
            "response":response,
            "reference":reference
        }
    )

Querying Pinecone Index: Done!
Decomposing Question: Done!
Generating QA Pairs: Done!
[('1. How many people attended the project meeting?', AIMessage(content='Four people attended the project meeting: Czech, Gian, Shaundyl, and Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 439, 'total_tokens': 458, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4-turbo-2024-04-09', 'system_fingerprint': 'fp_bf9cb2c77f', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2def394-3813-447b-9dfc-90ba2d078fbe-0', usage_metadata={'input_tokens': 439, 'output_tokens': 19, 'total_tokens': 458, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})), ('2. What were the names of the individuals who attended 

In [42]:
print(dataset)

[{'user_input': 'Who were the attendees of the project meeting', 'retrieved_contexts': ["Czech: Hello my name is Czech.\nGian: Hello my name is Gian.\nShaundyl: Hello my name is Shaundyl.\nCzech (Team Lead): Alright, everyone, thanks for joining today’s meeting. We have about 10 minutes to go over the final details before the product launch. Let's start with the progress update. Bob, how are we doing on the development front?", 'Gian: We’ve set up a dedicated support channel for the product and briefed the customer support team on the common issues we’re anticipating. We’ll also monitor social media for any unexpected feedback.\nCzech: Sounds like we’re in good shape. Thanks, everyone. Let’s aim to regroup tomorrow for a final status check. Anything else before we wrap up?\nShaundyl: Nothing from my side. I’ll update you if any blockers come up.\nGian: I’m all set. Let’s get this done!', 'Czech: Great to hear that. Gian, how are we looking on the project timeline? Any changes or concer

In [43]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper


evaluator_llm = LangchainLLMWrapper(LLM)
evaluator_embedding = LangchainEmbeddingsWrapper(EMBEDDINGS)

from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy, ContextPrecision

result = evaluate(dataset=evaluation_dataset,metrics=[LLMContextRecall(), Faithfulness(), FactualCorrectness(), ResponseRelevancy(), ContextPrecision()],llm=evaluator_llm, embeddings=evaluator_embedding)
result

Evaluating: 100%|██████████| 15/15 [00:44<00:00,  2.94s/it]


{'context_recall': 1.0000, 'faithfulness': 0.8000, 'factual_correctness': 0.5900}

In [55]:
os.environ["RAGAS_APP_TOKEN"] = "apt.4f8e-299f727bff54-c9b1-b0b4-f480bd0c-ea62a"

result.upload()

Evaluation results uploaded! View at https://app.ragas.io/dashboard/alignment/evaluation/b20ed325-a229-4b10-82a8-3beb28cb41c9


'https://app.ragas.io/dashboard/alignment/evaluation/b20ed325-a229-4b10-82a8-3beb28cb41c9'